In [1]:
using JuMP
using HiGHS
using Random

### Setup

In [2]:
function get_funds(n::Int64, seed::Int64=30)
    generator = MersenneTwister(seed)
    data = Dict() 
    for i in 1:n
        data["fund $i"] = Dict("expected return"   => rand(generator) > 0.5 ? -rand(generator) : rand(generator),
                               "annualized return" => rand(generator) > 0.5 ? -rand(generator) : rand(generator)
                          )
    end
    return data
end

get_funds (generic function with 2 methods)

In [3]:
funds = get_funds(50, 100);

In [4]:
__TOTAL_FUNDS__ = length(funds);

### Modeling

In [15]:
model = Model(HiGHS.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS

In [16]:
# Percentage of capital allocated to each fund
@variable(model, Allocation[key in keys(funds)] >= 0)

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, ["fund 40", "fund 42", "fund 50", "fund 19", "fund 10", "fund 12", "fund 35", "fund 48", "fund 25", "fund 1"  …  "fund 30", "fund 33", "fund 39", "fund 43", "fund 3", "fund 5", "fund 45", "fund 7", "fund 34", "fund 46"]
And data, a 50-element Vector{VariableRef}:
 Allocation[fund 40]
 Allocation[fund 42]
 Allocation[fund 50]
 Allocation[fund 19]
 Allocation[fund 10]
 Allocation[fund 12]
 Allocation[fund 35]
 Allocation[fund 48]
 Allocation[fund 25]
 Allocation[fund 1]
 Allocation[fund 28]
 Allocation[fund 17]
 Allocation[fund 21]
 ⋮
 Allocation[fund 38]
 Allocation[fund 27]
 Allocation[fund 30]
 Allocation[fund 33]
 Allocation[fund 39]
 Allocation[fund 43]
 Allocation[fund 3]
 Allocation[fund 5]
 Allocation[fund 45]
 Allocation[fund 7]
 Allocation[fund 34]
 Allocation[fund 46]

In [17]:
# Total allocated to all fund should equal 100%
@constraint(model, sum(Allocation) == 100)

Allocation[fund 40] + Allocation[fund 42] + Allocation[fund 50] + Allocation[fund 19] + Allocation[fund 10] + Allocation[fund 12] + Allocation[fund 35] + Allocation[fund 48] + Allocation[fund 25] + Allocation[fund 1] + Allocation[fund 28] + Allocation[fund 17] + Allocation[fund 21] + Allocation[fund 18] + Allocation[fund 11] + Allocation[fund 2] + Allocation[fund 23] + Allocation[fund 20] + Allocation[fund 15] + Allocation[fund 29] + Allocation[fund 26] + Allocation[fund 16] + Allocation[fund 37] + Allocation[fund 44] + Allocation[fund 22] + Allocation[fund 49] + Allocation[fund 31] + Allocation[fund 4] + Allocation[fund 13] + Allocation[fund 47] + Allocation[fund 8] + Allocation[fund 24] + Allocation[fund 32] + Allocation[fund 36] + Allocation[fund 6] + Allocation[fund 14] + Allocation[fund 41] + Allocation[fund 9] + Allocation[fund 38] + Allocation[fund 27] + Allocation[fund 30] + Allocation[fund 33] + Allocation[fund 39] + Allocation[fund 43] + Allocation[fund 3] + Allocation[fund 5

In [18]:
# No fund should receive more than 50% of available captial
@constraint(model, [fund in keys(funds)], Allocation[fund] <= 25)

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, ["fund 40", "fund 42", "fund 50", "fund 19", "fund 10", "fund 12", "fund 35", "fund 48", "fund 25", "fund 1"  …  "fund 30", "fund 33", "fund 39", "fund 43", "fund 3", "fund 5", "fund 45", "fund 7", "fund 34", "fund 46"]
And data, a 50-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 Allocation[fund 40] <= 25.0
 Allocation[fund 42] <= 25.0
 Allocation[fund 50] <= 25.0
 Allocation[fund 19] <= 25.0
 Allocation[fund 10] <= 25.0
 Allocation[fund 12] <= 25.0
 Allocation[fund 35] <= 25.0
 Allocation[fund 48] <= 25.0
 Allocation[fund 25] <= 25.0
 Allocation[fund 1] <= 25.0
 Allocation[fund 28] <= 25.0
 Allocation[fund 17] <= 25.0
 Allocation[fund 21] <= 25.

In [19]:
# Any fund must receive a minimum allocation greater than 0
@constraint(model, [fund in keys(funds)], Allocation[fund] >= 1.0)

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, ["fund 40", "fund 42", "fund 50", "fund 19", "fund 10", "fund 12", "fund 35", "fund 48", "fund 25", "fund 1"  …  "fund 30", "fund 33", "fund 39", "fund 43", "fund 3", "fund 5", "fund 45", "fund 7", "fund 34", "fund 46"]
And data, a 50-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 Allocation[fund 40] >= 1.0
 Allocation[fund 42] >= 1.0
 Allocation[fund 50] >= 1.0
 Allocation[fund 19] >= 1.0
 Allocation[fund 10] >= 1.0
 Allocation[fund 12] >= 1.0
 Allocation[fund 35] >= 1.0
 Allocation[fund 48] >= 1.0
 Allocation[fund 25] >= 1.0
 Allocation[fund 1] >= 1.0
 Allocation[fund 28] >= 1.0
 Allocation[fund 17] >= 1.0
 Allocation[fund 21] >= 1.0
 ⋮
 A

In [20]:
#M Maximize total return
@objective(model, Max, sum( (funds[fund]["expected return"] + funds[fund]["annualized return"]) * Allocation[fund] for fund in keys(funds)))

1.551033017992303 Allocation[fund 40] - 1.040201124186483 Allocation[fund 42] + 0.15198895044004757 Allocation[fund 50] + 0.702019596907856 Allocation[fund 19] - 0.0828566122156742 Allocation[fund 10] + 0.322389847665016 Allocation[fund 12] + 1.112804259178323 Allocation[fund 35] - 0.892220253538901 Allocation[fund 48] - 0.16065857660767602 Allocation[fund 25] + 0.12311964423142441 Allocation[fund 1] - 0.9839568566452981 Allocation[fund 28] - 1.43302600175762 Allocation[fund 17] + 1.0315180763072245 Allocation[fund 21] + 0.3015372681204509 Allocation[fund 18] + 0.45671452487601116 Allocation[fund 11] - 1.1725358858194055 Allocation[fund 2] - 1.0511712080095226 Allocation[fund 23] - 0.6884417490615553 Allocation[fund 20] - 1.4436231055944706 Allocation[fund 15] + 0.17384401326863275 Allocation[fund 29] - 0.36441384221945383 Allocation[fund 26] + 0.5419014081752187 Allocation[fund 16] - 0.3758357839350428 Allocation[fund 37] + 0.004358782992474719 Allocation[fund 44] + 1.0009835046763693

In [21]:
print(model)

In [22]:
optimize!(model)

Presolving model
1 rows, 50 cols, 50 nonzeros
1 rows, 50 cols, 50 nonzeros
Presolve : Reductions: rows 1(-100); columns 50(-0); elements 50(-100)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
          1    -8.7234542777e+01 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 1
Objective value     :  8.7234542777e+01
HiGHS run time      :          0.00


In [23]:
objective_value(model)

87.2345427770222

### Post-processing

In [24]:
using Printf
@printf("| %10s | %15s | %10s | %10s |\n", "Fund", "Proj. Return", "Annualized", "Allocation")
for fund in sort([fund for fund in keys(funds)])
    @printf("| %-10s | %15.4f | %10.4f | %10s |\n", 
        fund, 
        funds[fund]["expected return"], 
        funds[fund]["annualized return"],
        value(Allocation[fund])
    )
end

|       Fund |    Proj. Return | Annualized | Allocation |
| fund 1     |          0.1903 |    -0.0672 |        1.0 |
| fund 10    |         -0.0136 |    -0.0693 |        1.0 |
| fund 11    |          0.7654 |    -0.3087 |        1.0 |
| fund 12    |          0.6789 |    -0.3565 |        1.0 |
| fund 13    |         -0.2942 |     0.7588 |        1.0 |
| fund 14    |          0.2117 |    -0.8717 |        1.0 |
| fund 15    |         -0.6858 |    -0.7579 |        1.0 |
| fund 16    |         -0.3606 |     0.9025 |        1.0 |
| fund 17    |         -0.9250 |    -0.5081 |        1.0 |
| fund 18    |          0.0073 |     0.2942 |        1.0 |
| fund 19    |          0.7500 |    -0.0480 |        1.0 |
| fund 2     |         -0.6457 |    -0.5268 |        1.0 |
| fund 20    |          0.0727 |    -0.7612 |        1.0 |
| fund 21    |          0.5670 |     0.4645 |        1.0 |
| fund 22    |          0.6913 |     0.3096 |        1.0 |
| fund 23    |         -0.2320 |    -0.8192 |        1.0